# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [19]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: 0.09500000160187483
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [5]:
from collections import deque
import torch

target_avg_score = 0.5

def train(agents, env, brain_name, n_episodes=5000, print_every=100):
    
    scores_deque = deque(maxlen=print_every)
    scores = []
    moving_avg = []
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        agents[0].reset()                                      # initialize the agent
        agents[1].reset()
        episode_scores = np.zeros(num_agents)                  # initialize the score (for each agent)
        
        while True:
            agent0_action = agents[0].act(states[0])            # agents take actions
            agent1_action = agents[1].act(states[1])
            actions = np.vstack((agent0_action, agent1_action))
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agents[0].step(states[0], actions[0], rewards[0], next_states[0], dones)
            agents[1].step(states[1], actions[1], rewards[1], next_states[1], dones)
            episode_scores += rewards                          # update the score
            
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
                
        episode_avg_score = np.mean(episode_scores)
        scores.append(episode_avg_score)
        scores_deque.append(episode_avg_score)
        moving_avg.append(np.mean(scores_deque))
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        
        if np.mean(scores_deque) >= target_avg_score:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agents[0].actor_local.state_dict(), 'checkpoint_actor_0.pth')
            torch.save(agents[0].critic_local.state_dict(), 'checkpoint_critic_0.pth')
            torch.save(agents[1].actor_local.state_dict(), 'checkpoint_actor_1.pth')
            torch.save(agents[1].critic_local.state_dict(), 'checkpoint_critic_1.pth')
            break
        
        if i_episode % print_every == 0:
            print('\nEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
            
    return scores, moving_avg

import signal
from contextlib import contextmanager
import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable

In [ ]:
from src.ddpg_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

agents = [Agent(state_size, action_size) for _ in range(num_agents)]
with active_session():
    scores, moving_avg = train(agents, env, brain_name)
    
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='single episode mean score')
plt.plot(np.arange(len(moving_avg)), moving_avg, c='r', label='mean score over 100 episodes')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left');
plt.show()

Episode 100	Average Score: -0.00
Episode 200	Average Score: -0.00
Episode 300	Average Score: -0.00
Episode 400	Average Score: -0.00
Episode 500	Average Score: -0.00
Episode 600	Average Score: -0.00
Episode 700	Average Score: -0.00
Episode 800	Average Score: -0.00
Episode 900	Average Score: -0.00
Episode 1000	Average Score: -0.00
Episode 1100	Average Score: -0.00
Episode 1200	Average Score: -0.00
Episode 1300	Average Score: -0.00
Episode 1400	Average Score: -0.00
Episode 1500	Average Score: -0.00
Episode 1600	Average Score: -0.00
Episode 1700	Average Score: -0.00
Episode 1800	Average Score: -0.00
Episode 1900	Average Score: 0.000
Episode 2000	Average Score: 0.01
Episode 2100	Average Score: 0.01
Episode 2200	Average Score: 0.02
Episode 2300	Average Score: 0.03
Episode 2400	Average Score: 0.03
Episode 2500	Average Score: 0.04
Episode 2600	Average Score: 0.04
Episode 2700	Average Score: 0.04
Episode 2800	Average Score: 0.04
Episode 2900	Average Score: 0.04
Episode 3000	Average Score: 0.07


In [2]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import torch
from src.ddpg_agent import Agent

try: # Check if the env and agent has been initialized
    env
    agents
except:
    env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=False)[brain_name]
    num_agents = len(env_info.agents)
    action_size = brain.vector_action_space_size
    states = env_info.vector_observations
    state_size = states.shape[1]
    agents = [Agent(state_size, action_size) for _ in range(num_agents)]
    
    agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor_0.pth'))
    agents[0].critic_local.load_state_dict(torch.load('checkpoint_critic_0.pth'))
    agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor_1.pth'))
    agents[1].critic_local.load_state_dict(torch.load('checkpoint_critic_1.pth'))
    
for i in range(5):                                                 # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]             # reset the environment    
    states = env_info.vector_observations                          # get the current state (for each agent)
    scores = np.zeros(num_agents)                                  # initialize the score (for each agent)
    while True:
        agent0_action = agents[0].act(states[0], add_noise=False)  # select an action (for each agent)
        agent1_action = agents[1].act(states[1], add_noise=False)
        actions = np.vstack((agent0_action, agent1_action)) 
        env_info = env.step(actions)[brain_name]                   # send all actions to tne environment
        next_states = env_info.vector_observations                 # get next state (for each agent)
        rewards = env_info.rewards                                 # get reward (for each agent)
        dones = env_info.local_done                                # see if episode finished
        scores += env_info.rewards                                 # update the score (for each agent)
        states = next_states                                       # roll over states to next time step
        if np.any(dones):                                          # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Total score (averaged over agents) this episode: 0.3450000053271651
Total score (averaged over agents) this episode: 0.1450000023469329
Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: 0.3950000060722232
Total score (averaged over agents) this episode: 0.1450000023469329


When finished, you can close the environment.

In [ ]:
env.close()